In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240715'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/1.make.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Make No'] = input_['Make No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests
        import json

        import sys
        sys.path.append('../../00.Tools')
        from crawler_configuration import get_header, get_proxy

        payload = {'jsn': {'tab': 'catalog',
                           'make': input_.loc[a, 'Make'],
                           'nodetype': 'make'}}

        data = {'func': 'navnode_fetch',
                'payload': json.dumps(payload),
                'api_json_request': '1'}

        try:
            b = 0
            while True:

                while True:
                    b += 1
                    try:
                        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                             data=data,
                                             headers=get_header(),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))
                        break
                    except:
                        continue

                if resp.status_code == 200:
                    json_ = resp.json()
                    if 'redirect_to_url' in json_:
                        continue
                    elif json_['jsn']['tab'] == 'catalog' and json_['jsn']['make'] == input_.loc[a, 'Make'] and json_['jsn']['nodetype'] == 'make' and 'collected_javascript' in json_:
                        break

            # = = = = = = = = = = = = = = =

            from bs4 import BeautifulSoup
            from lxml import etree

            soup = BeautifulSoup(json_['html_fill_sections']['navchildren[]'], 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            list_json = html.xpath('//input[contains(@id, "jsn[__GIP__")]/@value')

            # = = = = = = = = = = = = = = =

            list_year = [int(json.loads(json_)['year'].strip()) for json_ in list_json]

            # = = = = = = = = = = = = = = =

            if not list_year:
                raise

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Make No': input_.loc[a, 'Make No'],
                                    'Make': input_.loc[a, 'Make'],
                                    'Year': list_year})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Make No': input_.loc[a, 'Make No'],
                                     'Make': input_.loc[a, 'Make']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Make No']}.{input_.loc[a, 'Make']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Make No', 'Year'],
                                                ascending=[True, False],
                                                ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/2.year-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Make No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'./{param_date}/year_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：271

[状态：ok，尝试次数：  1] - 2.AC
[剩余数量：251] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 13.AMPHICAR
[剩余数量：250] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 5.ALLARD
[剩余数量：249] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 20.AUBURN
[剩余数量：248] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 3.ACURA
[剩余数量：247] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 8.ALVIS
[剩余数量：246] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 7.ALPINE
[剩余数量：245] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 19.ASUNA
[剩余数量：244] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 15.ARMSTRONG-SIDDELEY
[剩余数量：243] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 12.AMERICAN MOTORS
[剩余数量：242] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 18.ASTON MARTIN
[剩余数量：241] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 17.ARNOLT-MG
[剩余数量：240] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 6.ALLSTATE
[剩余数量：239] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 10.AMERICAN AUSTIN
[剩余数量：238] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 1.ABARTH
[剩余数量：237] - [当前时间：09:59:05]

[状态：ok，尝试次数：  1] - 11.AMERICAN BANTAM
[剩余数量：236] - [当前时间：09:59:05]

[状态：ok，尝试

NameError: name 'rue' is not defined